In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# project_dir = "/content/drive/MyDrive/AraGenEval2025"
# os.chdir(project_dir)
# print(f"✅ Current directory: {os.getcwd()}")


In [3]:
import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) < 4)

!ls /kaggle/working/

MODEL_WEIGHTS_SRC ="/kaggle/input/ubc-nlp-rat5v2-base-1024-model/UBC-NLP_AraT5v2-base-1024/final"+"/*"

lora_injection_src= "/kaggle/input/ubc-ara-t5-v2-lora-ast-model/UBC-NLP_AraT5v2-base-1024_with_lora/UBC-NLP_AraT5v2-base-1024_with_lora/final"+"/*"

MODEL_NAME = "UBC-NLP_AraT5v2-base-1024"
# MODEL_NAME = "UBC-NLP/AraT5v2-base-1024" #"agemagician_mlong-t5-tglobal-large" #"facebook_mbart-large-50-many-to-many-mmt" # "UBC-NLP_AraT5-base" #"google/mt5-small"

MODEL_WEIGHTS_DEST_DIR =f"/kaggle/working/models/fine_tuned_model/model_weights/{MODEL_NAME}"

base_model_path = MODEL_WEIGHTS_DEST_DIR
lora_path       = f"./{MODEL_NAME}_with_lora"

# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    !mkdir -p {MODEL_WEIGHTS_DEST_DIR}
    !cp -r {MODEL_WEIGHTS_SRC} {MODEL_WEIGHTS_DEST_DIR}
    !mkdir -p {lora_path}
    !cp -r {lora_injection_src} {lora_path}
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")

# !mkdir -p {MODEL_WEIGHTS_DEST_DIR}
# !cp -r {MODEL_WEIGHTS_SRC} {MODEL_WEIGHTS_DEST_DIR}

# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

✅ '/kaggle/working' is empty. copying project strcuture into it ...
/kaggle/working
config.py  evaluation	models	 run_pipeline.py		      utils
data	   __init__.py	prompts  UBC-NLP_AraT5v2-base-1024_with_lora


In [4]:
!ls -R {MODEL_WEIGHTS_DEST_DIR}

/kaggle/working/models/fine_tuned_model/model_weights/UBC-NLP_AraT5v2-base-1024:
added_tokens.json	model.safetensors	 tokenizer_config.json
config.json		special_tokens_map.json  training_args.bin
generation_config.json	spiece.model


In [5]:
!ls -R {lora_path}

./UBC-NLP_AraT5v2-base-1024_with_lora:
adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json
added_tokens.json	   spiece.model


In [6]:
!pip install transformers[sentencepiece] torch pandas openpyxl psutil python-dotenv peft  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00:00:0100:01


In [7]:
# Ensure PyTorch can expand allocations if needed
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [8]:

import os
import json
import dotenv
import torch
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import gc
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from config import Config


# project root in PYTHONPATH
import sys
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"✅ Using device: {device}")

# Device & parallel setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}, count: {torch.cuda.device_count()} GPUs")

✅ Using device: cuda, count: 1 GPUs


In [9]:

def safe_model_name(model_name):
    return model_name.replace("/", "_")

# archive & inspect
def create_run_dir(base, name):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    d  = os.path.join(base, safe_model_name(name), f"run_{ts}")
    os.makedirs(d, exist_ok=True)
    return d

def run_evaluation():
  pass
    # os.system("python utils/prepare_metrics_log.py")
    # os.system("python evaluation/evaluate.py")


def archive_results(run_dir, files_to_archive):
    for fpath in files_to_archive:
        if os.path.exists(fpath):
            dest = os.path.join(run_dir, os.path.basename(fpath))
            os.system(f"cp {fpath} {dest}")


In [10]:
def load_model(model_name: str):
    secure = safe_model_name(model_name)
    model_path = os.path.join("models/fine_tuned_model/model_weights", secure)
    print(f"Loading model from {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    # mBART language tags
    if "bart" in model_name.lower():
        tokenizer.src_lang = tokenizer.tgt_lang = "ar_AR"
    # wrap for multi‑GPU
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    model.to(device).eval()
    return tokenizer, model  

In [11]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_lora_model(base_model_path: str, lora_path: str):
    print(f"Loading base model from {base_model_path}")
    print(f"Applying LoRA adapters from {lora_path}")

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False, legacy=False)

    # Base model
    base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_path)

    # Apply LoRA
    model = PeftModel.from_pretrained(base_model, lora_path)

    # Optional: for mBART-like models
    if "bart" in base_model_path.lower():
        tokenizer.src_lang = tokenizer.tgt_lang = "ar_AR"

    # DataParallel
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    model.to(device).eval()
    return tokenizer, model

2025-07-20 23:25:33.965469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753053934.167155      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753053934.229911      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
def save_predictions_to_file(records, df, output_log_path):
    # Map records by ID
    records_by_id = {r["id"]: r for r in records}

    # Re-order by original ID order
    ordered = [records_by_id[i] for i in df["id"] if i in records_by_id]
    missing = [i for i in df["id"] if i not in records_by_id]
    if missing:
        print(f"⚠️ Missing {len(missing)} records: {missing[:10]}...")

    # Save
    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(ordered, f, ensure_ascii=False, indent=2)
    print(f"✅ Generated {len(ordered)} records saved to {output_log_path}")


In [13]:
# ### Batched inference function (robust, with error recovery)
# def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
#     import time
#     import traceback

#     # 1. Load & prepare
#     df = pd.read_excel(validation_path)
#     df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

#     # Handle DataParallel wrapping
#     base_model = model.module if hasattr(model, "module") else model
#     base_model.generation_config.pad_token_id = tokenizer.pad_token_id
#     assert base_model.config.eos_token_id is not None

#     records = []

#     from torch.utils.data import DataLoader, Dataset

#     class TextDataset(Dataset):
#         def __init__(self, texts): self.texts = texts
#         def __len__(self): return len(self.texts)
#         def __getitem__(self, idx): return self.texts[idx]

#     def collate_fn(batch_texts):
#         return tokenizer(
#             batch_texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=1024
#         )

#     loader = DataLoader(
#         TextDataset(df["input_text"].tolist()),
#         batch_size=batch_size,
#         shuffle=False,
#         collate_fn=collate_fn,
#         num_workers=4,
#         pin_memory=True
#     )

#     for i, batch in enumerate(tqdm(loader, desc="Batches")):
#         success = False
#         attempts = 0
#         while not success and attempts < 3:
#             try:
#                 batch = {k: v.to(device) for k, v in batch.items()}
#                 with torch.inference_mode():
#                     gen_ids = base_model.generate(
#                         input_ids=batch["input_ids"],
#                         attention_mask=batch["attention_mask"],
#                         max_new_tokens=1024
#                     )
#                 outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

#                 start_idx = i * batch_size
#                 for j, out in enumerate(outs):
#                     row = df.iloc[start_idx + j]
#                     records.append({
#                         "id": int(row["id"]),
#                         "author": row["author"],
#                         "output": out,
#                         "ground_truth": row["text_in_author_style"]
#                     })
#                 success = True

#             except RuntimeError as e:
#                 attempts += 1
#                 print(f"❌ RuntimeError on batch {i}, attempt {attempts}: {str(e)}")
#                 traceback.print_exc()

#                 torch.cuda.empty_cache()
#                 gc.collect()

#                 if "CUDA out of memory" in str(e):
#                     print("⚠️ Reducing batch size temporarily and retrying...")
#                     time.sleep(2)
#                     if batch["input_ids"].shape[0] > 1:
#                         # Retry each sample individually
#                         for j in range(batch["input_ids"].shape[0]):
#                             try:
#                                 single_batch = {k: v[j:j+1] for k, v in batch.items()}
#                                 with torch.inference_mode():
#                                     gen_ids = base_model.generate(
#                                         input_ids=single_batch["input_ids"],
#                                         attention_mask=single_batch["attention_mask"],
#                                         max_new_tokens=1024
#                                     )
#                                 out = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
#                                 row = df.iloc[i * batch_size + j]
#                                 records.append({
#                                     "id": int(row["id"]),
#                                     "author": row["author"],
#                                     "output": out,
#                                     "ground_truth": row["text_in_author_style"]
#                                 })
#                             except Exception as inner_e:
#                                 print(f"⚠️ Skipping failed sample {i * batch_size + j} id = {int(row['id'])}: {str(inner_e)}")
#                                 traceback.print_exc()
#                     success = True  # Continue to next batch regardless
#                 else:
#                     raise e

#         torch.cuda.empty_cache()
#         gc.collect()

#     # 4. Save
#     with open(output_log_path, "w", encoding="utf-8") as f:
#         json.dump(records, f, ensure_ascii=False, indent=2)
#     print(f"✅ Generated {len(records)} records to {output_log_path}")


In [14]:
def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
    import time, gc, json, traceback
    import torch
    from torch.utils.data import DataLoader, Dataset
    from tqdm import tqdm
    import pandas as pd

    # 1. Load & prepare
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)
    
    # Compute token lengths
    df["token_len"] = df["input_text"].apply(lambda x: len(tokenizer.encode(x, truncation=True, max_length=1024)))
    
    # Save original order
    original_order = df[["id"]].copy()

    # Sort by token length
    df_sorted = df.sort_values("token_len").reset_index(drop=True)

    # Handle DataParallel wrapping
    base_model = model.module if hasattr(model, "module") else model
    base_model.generation_config.pad_token_id = tokenizer.pad_token_id
    assert base_model.config.eos_token_id is not None

    records = []

    class TextDataset(Dataset):
        def __init__(self, texts): self.texts = texts
        def __len__(self): return len(self.texts)
        def __getitem__(self, idx): return self.texts[idx]

    def collate_fn(batch_texts):
        return tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        )

    loader = DataLoader(
        TextDataset(df_sorted["input_text"].tolist()),
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True
    )

    for i, batch in enumerate(tqdm(loader, desc="Batches")):
        success = False
        attempts = 0
        while not success and attempts < 3:
            try:
                batch = {k: v.to(device) for k, v in batch.items()}
                with torch.inference_mode():
                    gen_ids = base_model.generate(
                        input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        max_new_tokens=1024
                    )
                outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

                start_idx = i * batch_size
                for j, out in enumerate(outs):
                    row = df_sorted.iloc[start_idx + j]
                    records.append({
                        "id": int(row["id"]),
                        "author": row["author"],
                        "output": out,
                        "ground_truth": row["text_in_author_style"]
                    })
                success = True

            except RuntimeError as e:
                attempts += 1
                print(f"❌ RuntimeError on batch {i}, attempt {attempts}: {str(e)}")
                traceback.print_exc()
                torch.cuda.empty_cache()
                gc.collect()

                if "CUDA out of memory" in str(e):
                    print("⚠️ Reducing batch size temporarily and retrying...")
                    time.sleep(2)
                    if batch["input_ids"].shape[0] > 1:
                        for j in range(batch["input_ids"].shape[0]):
                            try:
                                single_batch = {k: v[j:j+1] for k, v in batch.items()}
                                with torch.inference_mode():
                                    gen_ids = base_model.generate(
                                        input_ids=single_batch["input_ids"],
                                        attention_mask=single_batch["attention_mask"],
                                        max_new_tokens=1024
                                    )
                                out = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
                                row = df_sorted.iloc[i * batch_size + j]
                                records.append({
                                    "id": int(row["id"]),
                                    "author": row["author"],
                                    "output": out,
                                    "ground_truth": row["text_in_author_style"]
                                })
                            except Exception as inner_e:
                                print(f"⚠️ Skipping failed sample {i * batch_size + j} id = {int(row['id'])}: {str(inner_e)}")
                                traceback.print_exc()
                    success = True
                else:
                    raise e

        torch.cuda.empty_cache()
        gc.collect()

    # Restore original order
    df_records = pd.DataFrame(records)
    df_final = original_order.merge(df_records, on="id", how="left")

    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(df_final.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    print(f"✅ Generated {len(df_final)} records to {output_log_path}")


In [ ]:
def generate_predictions_by_budget(tokenizer, model, validation_path, output_log_path,
                         initial_token_budget=9_000, max_retries=3):
    import time, gc, json, traceback, torch
    from tqdm import tqdm
    import pandas as pd

    # 1. Load & prepare
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

    # Compute token lengths once
    df["tok_len"] = df["input_text"].apply(
        lambda x: len(tokenizer.encode(x, truncation=True, max_length=1024))
    )

    # Save original order
    original_order = df[["id"]].copy()

    # Sort by length ascending
    df_sorted = df.sort_values("tok_len").reset_index(drop=True)

    # Setup model
    base_model = model.module if hasattr(model, "module") else model
    base_model.generation_config.pad_token_id = tokenizer.pad_token_id
    device_ = next(base_model.parameters()).device

    records = []
    token_budget = initial_token_budget
    retry = 0

    while retry < max_retries:
        records.clear()
        try:
            batch_idxs = []
            current_sum = 0

            # build batches by token budget
            for idx, row in df_sorted.iterrows():
                ln = row["tok_len"]
                if ln > token_budget:
                    # single example too big: process alone
                    batch_idxs.append([idx])
                else:
                    if current_sum + ln <= token_budget:
                        batch_idxs[-1].append(idx) if batch_idxs else batch_idxs.append([idx])
                        current_sum += ln
                    else:
                        batch_idxs.append([idx])
                        current_sum = ln

            # run generation per batch
            for b in tqdm(batch_idxs, desc="Batches"):
                texts = df_sorted.loc[b, "input_text"].tolist()
                enc = tokenizer(texts, return_tensors="pt",
                                padding=True, truncation=True, max_length=1300)
                enc = {k: v.to(device_) for k, v in enc.items()}

                with torch.inference_mode():
                    gen = base_model.generate(
                        **enc,
                        max_new_tokens=1500,
                        use_cache=False
                    )
                outs = tokenizer.batch_decode(gen, skip_special_tokens=True)

                for idx, out in zip(b, outs):
                    row = df_sorted.loc[idx]
                    records.append({
                        "id":    int(row["id"]),
                        "author":row["author"],
                        "output":out,
                        "ground_truth": row["text_in_author_style"]
                    })

                torch.cuda.empty_cache()
                gc.collect()

            # success, break retry loop
            break

        except RuntimeError as e:
            if "out of memory" in str(e).lower() and retry < max_retries - 1:
                print(f"⚠️ OOM on token_budget={token_budget}, retrying with half budget")
                torch.cuda.empty_cache(); gc.collect()
                token_budget = token_budget // 2
                retry += 1
                time.sleep(2)
                continue
            else:
                raise

    # Restore original order
    df_rec = pd.DataFrame(records)
    df_final = original_order.merge(df_rec, on="id", how="left")

    # Save results
    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(df_final.to_dict(orient="records"),
                  f, ensure_ascii=False, indent=2)

    print(f"✅ Generated {len(df_final)} records (token_budget={token_budget}) → {output_log_path}")


In [ ]:
# !pip install accelerate
# accelerate config      # answer prompts (choose multi‑GPU)

In [ ]:
# import os
# from config import Config  # import your custom Config module
# import pandas as pd
# import torch
# gc = __import__('gc')
# from tqdm import tqdm
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from accelerate import Accelerator
# import pandas as pd
# import torch
# gc = __import__('gc')
# from tqdm import tqdm
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from accelerate import Accelerator

# def generate_predictions(
#     tokenizer, model, validation_path, output_log_path,
#     max_new_tokens=1700,
#     token_budget=100_000,
#     sort_inputs=True
# ):
#     # Initialize Accelerator
#     accelerator = Accelerator()
#     # Prepare model & tokenizer
#     model, tokenizer = accelerator.prepare(model, tokenizer)
#     model.eval()
#     unwrapped = accelerator.unwrap_model(model)

#     # 1. Load & prepare DataFrame
#     df = pd.read_excel(validation_path)
#     df['input_text'] = df['author'].astype(str) + ' [SEP] ' + df['text_in_msa'].astype(str)

#     # 2. Compute token lengths
#     max_len = 1024 #unwrapped.config.max_position_embeddings
#     enc = tokenizer(
#         df['input_text'].tolist(), return_length=True,
#         padding=False, truncation=True, max_length=max_len
#     )
#     lengths = enc['length']

#     # 3. Build token-budgeted batches
#     indices = list(range(len(lengths)))
#     if sort_inputs:
#         indices.sort(key=lambda i: lengths[i], reverse=True)
#     batches, curr, curr_sum = [], [], 0
#     for i in indices:
#         cost = lengths[i] + max_new_tokens
#         if curr and curr_sum + cost > token_budget:
#             batches.append(curr)
#             curr, curr_sum = [], 0
#         curr.append(i)
#         curr_sum += cost
#     if curr:
#         batches.append(curr)

#     # 4. Generation loop with instrumentation
#     records = []
#     for batch_idx, batch_ids in enumerate(tqdm(batches, desc='Accelerated Batches')):
#         # Instrumentation
#         sample_count = len(batch_ids)
#         batch_token_cost = sum(lengths[i] + max_new_tokens for i in batch_ids)
#         print(f"\n▶ Batch {batch_idx+1}/{len(batches)}: samples={sample_count}, token_cost={batch_token_cost}")
#         # Memory before
#         for dev in range(torch.cuda.device_count()):
#             alloc = torch.cuda.memory_allocated(dev) / 1024**2
#             reserved = torch.cuda.memory_reserved(dev) / 1024**2
#             print(f"   • GPU {dev}: allocated={alloc:.1f} MiB, reserved={reserved:.1f} MiB")

#         texts = [df.iloc[i]['input_text'] for i in batch_ids]
#         inputs = tokenizer(
#             texts, return_tensors='pt', padding=True,
#             truncation=True, max_length=max_len
#         )
#         inputs = {k: v.to(accelerator.device) for k, v in inputs.items()}

#         with torch.no_grad():
#             gen_ids = model.generate(
#                 input_ids=inputs['input_ids'],
#                 attention_mask=inputs['attention_mask'],
#                 max_new_tokens=max_new_tokens
#             )
#         gen_ids = accelerator.gather(gen_ids)
#         outs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

#         # Memory after
#         for dev in range(torch.cuda.device_count()):
#             alloc = torch.cuda.memory_allocated(dev) / 1024**2
#             reserved = torch.cuda.memory_reserved(dev) / 1024**2
#             print(f"   • GPU {dev}: allocated={alloc:.1f} MiB, reserved={reserved:.1f} MiB")

#         for idx, out in zip(batch_ids, outs):
#             row = df.iloc[idx]
#             records.append({
#                 'id': int(row['id']),
#                 'author': row['author'],
#                 'output': out,
#                 'ground_truth': row['text_in_author_style']
#             })

#         # cleanup
#         del inputs, gen_ids
#         gc.collect()
#         torch.cuda.empty_cache()

#     # 5. Save results
#     import json
#     with open(output_log_path, 'w', encoding='utf-8') as f:
#         json.dump(records, f, ensure_ascii=False, indent=2)
#     print(f"✅ Generated {len(records)} records and saved to {output_log_path}")
#     return records, df



In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

In [ ]:
!nvidia-smi

In [ ]:
# ---------------- Cell 4: run the pipeline ----------------

# if __name__ == '__main__':
# adjust these paths as needed
validation_file = Config.VAL_FILE
log_file = f"models/fine_tuned_model/results/{safe_model_name(MODEL_NAME)}/generation_log.json"
output_base_dir = "analysis/fine_tuned_model"

os.makedirs(os.path.dirname(log_file), exist_ok=True)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# tokenizer, model = load_model(MODEL_NAME)

# base_model_path = "models/base_models/facebook/arabart"
# lora_path       = "models/fine_tuned_model/model_weights/lora_moussa_arabart"

tokenizer, model = load_lora_model(base_model_path, lora_path)


In [ ]:
# generate_predictions(tokenizer, model, validation_file, log_file, batch_size=48)
generate_predictions_by_budget(tokenizer, model, validation_file, log_file)

# records, df = generate_predictions(tokenizer, model, validation_file, log_file)
# If saving separately is needed
# save_predictions_to_file(records, df, log_file)

run_dir = create_run_dir(output_base_dir, MODEL_NAME)
os.system(f"cp {log_file} {run_dir}/")
print(f"\n✅ Pipeline done. Logs in: {run_dir} \n LOG FILE: {log_file}\n")

In [ ]:
!nvidia-smi

In [ ]:
from IPython.display import FileLink

# Display a link to the copied log file in the run_dir
log_file_in_run_dir = os.path.join(run_dir, os.path.basename(log_file))
display(FileLink(log_file_in_run_dir))

In [ ]:
# print(len(records))

In [ ]:
# # Step 1: Delete large objects
# # del model, tokenizer

# # # Step 2: If model is still in GPU, move to CPU first
# # model.to("cpu")  
# # # then 
# # del model

# # Step 3: Run GC and empty cache
# import gc, torch
# gc.collect()
# torch.cuda.empty_cache()

# # Step 4: Confirm
# print(f"Allocated: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
# print(f"Cached:    {torch.cuda.memory_reserved() / 1e6:.1f} MB")
